Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

In [2]:
import os
import sys

# Define your paths
paths = [
    r"C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\models",
    r"C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\models\research",
    r"C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\models\research\slim"
]

# Initialize PYTHONPATH if it doesn't exist
if 'PYTHONPATH' not in os.environ:
    os.environ['PYTHONPATH'] = ""

# Add paths to PYTHONPATH
os.environ['PYTHONPATH'] += ";" + ";".join(paths)

# Add paths to sys.path for Jupyter notebook runtime
for p in paths:
    if p not in sys.path:
        sys.path.append(p)

print("✅ PYTHONPATH and sys.path updated successfully!")


✅ PYTHONPATH and sys.path updated successfully!


# 0. Setup Paths

In [4]:
WORKSPACE_PATH = r'C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\workspace'
SCRIPTS_PATH = r'C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\scripts'
APIMODEL_PATH = r'C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\models'
ANNOTATION_PATH = WORKSPACE_PATH + r'\annotations'
IMAGE_PATH = WORKSPACE_PATH + r'\images'
MODEL_PATH = WORKSPACE_PATH + r'\models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + r'\pre-trained-models'
CONFIG_PATH = MODEL_PATH + r'\my_ssd_mobnet\pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + r'\my_ssd_mobnet'


# 1. Create Label Map

In [6]:
labels = [
    {'name': 'hello', 'id': 1},
    {'name': 'yes', 'id': 2},
    {'name': 'no', 'id': 3},
    {'name': 'thanks', 'id': 4},
    {'name': 'i love you', 'id': 5},
    {'name': 'help', 'id': 6},
    {'name': 'together', 'id': 7},
    {'name': 'good luck', 'id': 8},
    {'name': 'good', 'id': 9},
    {'name': 'please', 'id': 10},
    {'name': 'hope', 'id': 11},
    {'name': 'danger', 'id': 12}
]


with open(ANNOTATION_PATH + r'\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item {\n')
        f.write('  id: {}\n'.format(label['id']))
        f.write('  name: "{}"\n'.format(label['name']))
        f.write('}\n\n')


# 2. Create TF records

In [8]:
!python "{SCRIPTS_PATH}\\generate_tfrecord.py" -x "{IMAGE_PATH}\\train" -l "{ANNOTATION_PATH}\\label_map.pbtxt" -o "{ANNOTATION_PATH}\\train.record"

!python "{SCRIPTS_PATH}\\generate_tfrecord.py" -x "{IMAGE_PATH}\\test" -l "{ANNOTATION_PATH}\\label_map.pbtxt" -o "{ANNOTATION_PATH}\\test.record"


Successfully created the TFRecord file: C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\workspace\annotations\\train.record


2025-03-21 10:56:33.330760: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 10:56:37.072985: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Successfully created the TFRecord file: C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\workspace\annotations\\test.record


2025-03-21 10:56:56.702532: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 10:56:58.269663: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [10]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [11]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [13]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [14]:
import os
import shutil

CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_PATH = 'Tensorflow/workspace/pre-trained-models'
MODEL_PATH = 'Tensorflow/workspace/models'

# Create model directory
os.makedirs(os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME), exist_ok=True)

# Copy pipeline.config
shutil.copy(
    os.path.join(PRETRAINED_MODEL_PATH, 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', 'pipeline.config'),
    os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME)
)


'Tensorflow/workspace/models\\my_ssd_mobnet\\pipeline.config'

# 5. Update Config For Transfer Learning

In [16]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [17]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [18]:
from object_detection.utils import config_util
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [19]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [20]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [21]:
pipeline_config.model.ssd.num_classes = 12
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [22]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [24]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python C:\Users\khema\RealTimeObjectDetection-main\Tensorflow\models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 7. Load Train Model From Checkpoint

In [26]:
import os
os.environ['PYTHONPATH'] = (
    "C:/Users/khema/RealTimeObjectDetection-main/Tensorflow/models;"
    "C:/Users/khema/RealTimeObjectDetection-main/Tensorflow/models/research;"
    "C:/Users/khema/RealTimeObjectDetection-main/Tensorflow/models/research/slim"
)
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [27]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [29]:
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util, visualization_utils as viz_utils
from PIL import Image, ImageFont, ImageDraw

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Language selection (Default: English)
languages = {'E': 'English', 'T': 'Telugu', 'H': 'Hindi'}
selected_language = 'E'

# Translations for detected gestures
translations = {
    "hello": {"E": "Hello", "T": "హలో", "H": "नमस्ते"},
    "yes": {"E": "Yes", "T": "అవును", "H": "हाँ"},
    "no": {"E": "No", "T": "కాదు", "H": "नहीं"},
    "thanks": {"E": "Thanks", "T": "ధన్యవాదాలు", "H": "धन्यवाद"},
    "i love you": {"E": "I love you", "T": "నేను నిన్ను ప్రేమిస్తున్నాను", "H": "मैं तुमसे प्यार करता हूँ"},
    "help": {"E": "Help", "T": "సహాయం", "H": "मदद"},
    "together": {"E": "Together", "T": "కలిసి", "H": "साथ"},
    "good luck": {"E": "Good luck", "T": "శుభవార్త", "H": "शुभकामनाएँ"},
    "good": {"E": "Good", "T": "మంచి", "H": "अच्छा"},
    "please": {"E": "Please", "T": "దయచేసి", "H": "कृपया"},
    "hope": {"E": "Hope", "T": "ఆశ", "H": "आशा"},
    "danger": {"E": "Danger", "T": "ప్రమాదం", "H": "खतरा"}
}

# Load fonts
telugu_font_path = r"C:\Users\khema\RealTimeObjectDetection-main\fonts\NotoSansTelugu-VariableFont_wdth,wght.ttf"
hindi_font_path = r"C:\Users\khema\RealTimeObjectDetection-main\fonts\NotoSansDevanagari-VariableFont_wdth,wght.ttf"

english_font = ImageFont.truetype("arial.ttf", 40)  # Windows default font
telugu_font = ImageFont.truetype(telugu_font_path, 40)
hindi_font = ImageFont.truetype(hindi_font_path, 40)

# Dictionary to map language to font
fonts = {"E": english_font, "T": telugu_font, "H": hindi_font}

while True:
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    # Get detected class
    detected_class_id = detections['detection_classes'][0] + label_id_offset
    detected_class_name = category_index[detected_class_id]['name']
    
    # Get translation based on selected language
    translated_text = translations.get(detected_class_name, {}).get(selected_language, detected_class_name)

    # Convert OpenCV image to PIL
    image_pil = Image.fromarray(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(image_pil)

    # Display selected language
    draw.text((10, 30), f"Language: {languages[selected_language]}", font=english_font, fill=(0, 255, 0))

    # Display detected text in the selected language
    draw.text((10, height - 60), f"Detected: {translated_text}", font=fonts[selected_language], fill=(0, 255, 0))

    # Convert PIL image back to OpenCV format
    image_np_with_detections = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

    cv2.imshow('Sign Language Detection', cv2.resize(image_np_with_detections, (800, 600)))

    # Keypress handling for switching languages
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        cap.release()
        break
    elif key in [ord('E'), ord('e')]:
        selected_language = 'E'
    elif key in [ord('T'), ord('t')]:
        selected_language = 'T'
    elif key in [ord('H'), ord('h')]:
        selected_language = 'H'


In [32]:
import cv2 
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util, visualization_utils as viz_utils

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True: 
    ret, frame = cap.read()
    if not ret:
        break

    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    # Get detected sign with highest confidence
    max_score_idx = np.argmax(detections['detection_scores'])
    detected_class_id = detections['detection_classes'][max_score_idx] + label_id_offset
    detected_score = detections['detection_scores'][max_score_idx]

    detected_sign = category_index[detected_class_id]['name'] if detected_score > 0.5 else ""

    # Draw detection boxes but without labels
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.5,
        agnostic_mode=False,
        line_thickness=3,
        skip_labels=True,  # Skip labels from the bounding box
    )

    # Create chat box at the bottom
    overlay = image_np_with_detections.copy()
    cv2.rectangle(overlay, (0, height - 50), (width, height), (0, 0, 0), -1)  # Black rectangle
    alpha = 0.5  # Transparency level
    cv2.addWeighted(overlay, alpha, image_np_with_detections, 1 - alpha, 0, image_np_with_detections)

    # Display detected sign in chat box
    if detected_sign:
        cv2.putText(image_np_with_detections, f"Detected: {detected_sign}", (20, height - 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Sign Language Detection', cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
